<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From 922fa9556f88251d3ecb14e5bb217c5c9f57bd80 Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Mon, 5 Jul 2021 15:57:50 +0800
Subject: [PATCH] fix java aar inference bug

---
 mindspore/lite/java/native/CMakeLists.txt          |  2 ++
 .../lite/java/native/runtime/lite_session.cpp      | 21 ------------
 .../lite/java/native/runtime/train_session.cpp     | 39 ++++++++++++++++++++++
 3 files changed, 41 insertions(+), 21 deletions(-)
 create mode 100644 mindspore/lite/java/native/runtime/train_session.cpp

diff --git a/mindspore/lite/java/native/CMakeLists.txt b/mindspore/lite/java/native/CMakeLists.txt
index 2353b1d..5892831 100644
--- a/mindspore/lite/java/native/CMakeLists.txt
+++ b/mindspore/lite/java/native/CMakeLists.txt
@@ -103,6 +103,8 @@ endif()
 
 if(SUPPORT_TRAIN)
   set(LITE_TRAIN_SO_NAME mindspore-lite-train minddata-lite)
+  set(JNI_TRAIN_SRC ${CMAKE_CURRENT_SOURCE_DIR}/runtime/train_session.cpp)
+  add_library(mindspore-lite-jni SHARED ${JNI_TRAIN_SRC})
   if(PLATFORM_ARM64 OR PLATFORM_ARM32)
     find_library(log-lib log)
     target_link_libraries(mindspore-lite-jni ${LITE_TRAIN_SO_NAME} ${log-lib})
diff --git a/mindspore/lite/java/native/runtime/lite_session.cpp b/mindspore/lite/java/native/runtime/lite_session.cpp
index 0d2af8a..fb31598 100644
--- a/mindspore/lite/java/native/runtime/lite_session.cpp
+++ b/mindspore/lite/java/native/runtime/lite_session.cpp
@@ -411,24 +411,3 @@ extern "C" JNIEXPORT jobject JNICALL Java_com_mindspore_lite_LiteSession_getFeat
   }
   return ret;
 }
-
-extern "C" JNIEXPORT jlong JNICALL Java_com_mindspore_lite_LiteSession_createTrainSession(JNIEnv *env, jobject thiz,
-                                                                                          jstring file_name,
-                                                                                          jlong ms_context_ptr,
-                                                                                          jboolean train_mode,
-                                                                                          jlong train_config_ptr) {
-  auto *pointer = reinterpret_cast<void *>(ms_context_ptr);
-  if (pointer == nullptr) {
-    MS_LOGE("Context pointer from java is nullptr");
-    return jlong(nullptr);
-  }
-  auto *lite_context_ptr = static_cast<mindspore::lite::Context *>(pointer);
-
-  auto session = mindspore::session::LiteSession::CreateTrainSession(env->GetStringUTFChars(file_name, JNI_FALSE),
-                                                                     lite_context_ptr, train_mode, nullptr);
-  if (session == nullptr) {
-    MS_LOGE("CreateTrainSession failed");
-    return jlong(nullptr);
-  }
-  return jlong(session);
-}
diff --git a/mindspore/lite/java/native/runtime/train_session.cpp b/mindspore/lite/java/native/runtime/train_session.cpp
new file mode 100644
index 0000000..a799789
--- /dev/null
+++ b/mindspore/lite/java/native/runtime/train_session.cpp
@@ -0,0 +1,39 @@
+/**
+ * Copyright 2020 Huawei Technologies Co., Ltd
+ *
+ * Licensed under the Apache License, Version 2.0 (the "License");
+ * you may not use this file except in compliance with the License.
+ * You may obtain a copy of the License at
+ *
+ * http://www.apache.org/licenses/LICENSE-2.0
+ *
+ * Unless required by applicable law or agreed to in writing, software
+ * distributed under the License is distributed on an "AS IS" BASIS,
+ * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
+ * See the License for the specific language governing permissions and
+ * limitations under the License.
+ */
+
+#include <jni.h>
+#include "common/ms_log.h"
+#include "include/lite_session.h"
+#include "include/train/train_cfg.h"
+#include "include/errorcode.h"
+
+extern "C" JNIEXPORT jlong JNICALL Java_com_mindspore_lite_LiteSession_createTrainSession(
+  JNIEnv *env, jobject thiz, jstring file_name, jlong ms_context_ptr, jboolean train_mode, jlong train_config_ptr) {
+  auto *pointer = reinterpret_cast<void *>(ms_context_ptr);
+  if (pointer == nullptr) {
+    MS_LOGE("Context pointer from java is nullptr");
+    return jlong(nullptr);
+  }
+  auto *lite_context_ptr = static_cast<mindspore::lite::Context *>(pointer);
+
+  auto session = mindspore::session::LiteSession::CreateTrainSession(env->GetStringUTFChars(file_name, JNI_FALSE),
+                                                                     lite_context_ptr, train_mode, nullptr);
+  if (session == nullptr) {
+    MS_LOGE("CreateSession failed");
+    return jlong(nullptr);
+  }
+  return jlong(session);
+}
\ No newline at end of file
-- 
2.7.4

